#06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [43]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [3]:
# 2015-summary.json

In [4]:
df = spark.read.format('json').load("learning_spark_data")

In [5]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('_corrupt_record', 'string'),
 ('count', 'bigint')]

In [6]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)
 |-- count: long (nullable = true)



In [7]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', _corrupt_record=None, count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', _corrupt_record=None, count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', _corrupt_record=None, count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', _corrupt_record=None, count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', _corrupt_record=None, count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', _corrupt_record=None, count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', _corrupt_record=None, count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', _corrupt_record=None, count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', _corrupt_record=None, count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United S

In [8]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', _corrupt_record=None, count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', _corrupt_record=None, count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', _corrupt_record=None, count=344)]

In [9]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [17]:
# 도착 국가명 중복제거
df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [19]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()   # 캐시를 사용해 (메모리에 올려놓고 계속 써)
df1.count()

133

In [20]:
# ROW class를 이용한 단일 레코드 생성
from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [22]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr
df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME'))   # expr은 sql 표현식을 받아 생성 -> 즉, 같은 국가일 경우 withinCountry컬럼에 추가

In [23]:
df3.show(3)

+-----------------+-------------------+---------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|_corrupt_record|count|withinCountry|
+-----------------+-------------------+---------------+-----+-------------+
|    United States|            Romania|           NULL|   15|        false|
|    United States|            Croatia|           NULL|    1|        false|
|    United States|            Ireland|           NULL|  344|        false|
+-----------------+-------------------+---------------+-----+-------------+
only showing top 3 rows



In [30]:
# withinCountry가 true인 국가 뽑기
df3.filter(df3.withinCountry == True).show()

+-----------------+-------------------+---------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|_corrupt_record| count|withinCountry|
+-----------------+-------------------+---------------+------+-------------+
|    United States|      United States|           NULL|370002|         true|
+-----------------+-------------------+---------------+------+-------------+



In [35]:
# case when 이용해서 count가 10이하이면 under, 이상이면 upper로 변환 > category 컬럼 추가
df4 = df.withColumn('Category', expr("CASE WHEN count<10 THEN 'under' WHEN count>=10 THEN 'upper' END"))
df4.show(10)

+-----------------+-------------------+---------------+-----+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|_corrupt_record|count|Category|
+-----------------+-------------------+---------------+-----+--------+
|    United States|            Romania|           NULL|   15|   upper|
|    United States|            Croatia|           NULL|    1|   under|
|    United States|            Ireland|           NULL|  344|   upper|
|            Egypt|      United States|           NULL|   15|   upper|
|    United States|              India|           NULL|   62|   upper|
|    United States|          Singapore|           NULL|    1|   under|
|    United States|            Grenada|           NULL|   62|   upper|
|       Costa Rica|      United States|           NULL|  588|   upper|
|          Senegal|      United States|           NULL|   40|   upper|
|          Moldova|      United States|           NULL|    1|   under|
+-----------------+-------------------+---------------+-----+--------+
only s

In [36]:
# DataFrame의 select(). where(). filter() 트랜스포메이션
# show(), count() 액션

In [37]:
spark.stop()

In [38]:
# 집계함수
# emp_df, dept_df

In [57]:
#emp_df, dept_df
emp_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/emp.csv')
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [58]:
emp_df.show(5)

+-----+------+--------+----+----------+----+----+------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+------+--------+----+----------+----+----+------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES| MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN|SALESMAN|7698|1981-09-28|1250|1400|    30|
+-----+------+--------+----+----------+----+----+------+
only showing top 5 rows



In [59]:
dept_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/dept.csv')
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [60]:
dept_df.show(5)

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [62]:
# 대소문자 구분 X
emp_df.select('ENAME', 'DEPTNO').show()

+------+------+
| ENAME|DEPTNO|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [63]:
# filter()와 동일
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [64]:
emp_df.selectExpr('count(*)').show()   # 문자열로 count는 sql

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [65]:
from pyspark.sql.functions import countDistinct

emp_df.select( countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [66]:
from pyspark.sql.functions import approx_count_distinct

emp_df.select( approx_count_distinct('job', 0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [67]:
# first, last, min, max, sum, avg를 -> (Expr로(sql 문장) 해보고,) function으로 처리

In [69]:
from pyspark.sql.functions import first

emp_df.select( first('sal') ).show()

+----------+
|first(sal)|
+----------+
|       800|
+----------+



In [70]:
from pyspark.sql.functions import last

emp_df.select( last('sal') ).show()

+---------+
|last(sal)|
+---------+
|     3200|
+---------+



In [71]:
from pyspark.sql.functions import min

emp_df.select( min('sal') ).show()

+--------+
|min(sal)|
+--------+
|     800|
+--------+



In [72]:
from pyspark.sql.functions import max

emp_df.select( max('sal') ).show()

+--------+
|max(sal)|
+--------+
|    5000|
+--------+



In [73]:
from pyspark.sql.functions import sum

emp_df.select( sum('sal') ).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [74]:
from pyspark.sql.functions import avg

emp_df.select( avg('sal') ).show()

+------------------+
|          avg(sal)|
+------------------+
|2148.3333333333335|
+------------------+



In [76]:
# total_salaty / total_transaction, avg_salary, mean_salaty 비교

In [80]:
from pyspark.sql.functions import count, mean

In [81]:
emp_df.select(
    count('sal').alias('total_transaction'),
    sum('sal').alias('total_salary'),
    avg('sal').alias('avg_salary'),
    mean('sal').alias('mean_salary')
).selectExpr(
    'total_salary/total_transaction',
    'avg_salary',
    'mean_salary'
).show()

+----------------------------------+------------------+------------------+
|(total_salary / total_transaction)|        avg_salary|       mean_salary|
+----------------------------------+------------------+------------------+
|                2148.3333333333335|2148.3333333333335|2148.3333333333335|
+----------------------------------+------------------+------------------+



In [82]:
# 그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [84]:
#select job,
#    count(job),
#    sum(sal)
#group by job

group_df = emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
)
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [85]:
# sal의 평균(SAL_AVG), 표준편차를(SAL_STDEV) job별로 계산해서 출력, 소수점2자리까지

In [92]:
from pyspark.sql.functions import std, round

In [94]:
emp_df.groupBy('job').agg(
    round(avg('sal'), 2).alias('SAL_AVG'),
    round(std('sal'), 2).alias('SAL_STDEV')
).show()

+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [91]:
# 급여 TOP10 구하기

In [98]:
from pyspark.sql.functions import desc

In [99]:
emp_df.orderBy(desc('sal')).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [100]:
# 강사님 답 : 윈도우 함수

In [102]:
from pyspark.sql.functions import desc, rank

In [111]:
from pyspark.sql.window import Window
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [112]:
emp_df.withColumn('salary_rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [114]:
# 직무별로 rank 작성
# Window.partitionBy()
# job_rank_df 작성

In [116]:
windowspec_job = Window.partitionBy('job').orderBy(desc('sal'))
job_rank = rank().over(windowspec_job)

job_rank_df = emp_df.withColumn('job_rank', job_rank)
job_rank_df.show()

+-----+------+---------+----+----------+----+----+------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|job_rank|
+-----+------+---------+----+----------+----+----+------+--------+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|       1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|       1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|       1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|       2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|       3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|       4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|       5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|       1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|       2|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|       3|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|       1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|     

In [119]:
# 부서별 연봉 순위

In [120]:
windowspec_dept = Window.partitionBy('deptno').orderBy(desc('sal'))

dept_rank = rank().over(windowspec_dept)

dept_rank_df = emp_df.withColumn('dept_rank', dept_rank)
dept_rank_df.show()

+-----+------+---------+----+----------+----+----+------+---------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|dept_rank|
+-----+------+---------+----+----------+----+----+------+---------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        1|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        2|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        3|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|        1|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        3|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        1|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        2|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|        3|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 50

In [ ]:
# 부서별 누적급여sum('sal').over

In [121]:
dept_cum_sal_df = emp_df.withColumn('sal_cum_sum', sum('sal').over(windowspec_dept))

dept_cum_sal_df.show()

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|sal_cum_sum|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|       5000|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|       7450|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|       8750|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|       6000|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|       6000|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|       8975|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|      10075|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|      10875|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|       2850|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|       4450|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|       5950|
| 7521|  WARD| SALES

In [ ]:
# 부서별 평균급여와 직원 개별 급여 비교

In [124]:
from pyspark.sql.functions import col

In [125]:
windowspec_dept1 = Window.partitionBy('deptno')

emp_with_avg_sal_df = emp_df.withColumn('dept_avg_sal', avg('sal').over(windowspec_dept1))

final_df = emp_with_avg_sal_df.withColumn(
    'sal_vs_avg',
    col('sal') > col('dept_avg_sal')
)

# 결과 출력
final_df.show()

+-----+------+---------+----+----------+----+----+------+------------------+----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|      dept_avg_sal|sal_vs_avg|
+-----+------+---------+----+----------+----+----+------+------------------+----------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|2916.6666666666665|     false|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|2916.6666666666665|      true|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|2916.6666666666665|     false|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|            2175.0|     false|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|            2175.0|      true|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|            2175.0|      true|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|            2175.0|     false|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|            2175.0|      true|
| 7499| ALLEN| SALESMAN|7698|198

In [126]:
# 부서별 직업별 소개

In [127]:
emp_df.groupBy('deptno', 'job').agg(count('*'), sum('sal'))\
    .orderBy('deptno', 'job').show(5)

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
+------+---------+--------+--------+
only showing top 5 rows



In [129]:
emp_df.cube('deptno', 'job').agg(count('*'), sum('sal'))\
    .orderBy('deptno', 'job').show(5)   # 평균급, 최대급, 최소급

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
+------+---------+--------+--------+
only showing top 5 rows



In [130]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [131]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [132]:
join_df = emp_df.join(dept_df, on='deptno', how='inner')
join_df.select('ename', 'deptno', 'dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+

